##### Installing libraries

In [1]:
!pip install Scrapy
!pip install time
!pip install datetime
!pip install pandas

     |████████████████████████████████| 237 kB 1.3 MB/s eta 0:00:01
     |████████████████████████████████| 53 kB 1.5 MB/s eta 0:00:01
     |████████████████████████████████| 2.6 MB 3.4 MB/s eta 0:00:01     |███████████████████████████     | 2.2 MB 2.3 MB/s eta 0:00:01
     |████████████████████████████████| 3.2 MB 2.2 MB/s eta 0:00:01
     |████████████████████████████████| 3.1 MB 2.0 MB/s eta 0:00:01
  Using cached cffi-1.14.2-cp37-cp37m-manylinux1_x86_64.whl (401 kB)
     |████████████████████████████████| 52 kB 579 kB/s eta 0:00:011
     |████████████████████████████████| 48 kB 4.6 MB/s eta 0:00:011
  Using cached pycparser-2.20-py2.py3-none-any.whl (112 kB)
  Created wheel for PyDispatcher: filename=PyDispatcher-2.0.5-py3-none-any.whl size=11515 sha256=3a0ccb5e3e276160aa9fc3897db1244bfa2001941ce7bceb0338f1debfef60f1
  Stored in directory: /home/iazzari/.cache/pip/wheels/dc/d0/bf/0cc715c01fce0bace63b46283acf5cc630d5e5dbb4602c54e5
  Created wheel for protego: filename=Protego-0.1.16

##### Calling libraries

In [2]:
import requests
from scrapy.http import TextResponse

In [3]:
scrapy.Request(url='https://br.investing.com/search/?q=BRML3&tab=quotes')

<GET https://br.investing.com/search/?q=BRML3&tab=quotes>

In [33]:
res = requests.get('https://br.investing.com/')
response = TextResponse(res.url, body=res.text, encoding='utf-8')

# notice = 
response.xpath('/html/body/div[5]/section/div[1]/div[1]/div[1]/div[1]/article/div[1]/a').extract_first()


# print(notice.text)

In [44]:
res = requests.get('https://tecnoblog.net/')

# print(response.xpath("/html/body/div[6]/div/div/div[2]/div[1]/div[3]/div[1]/a/h2").extract_first().text)

for i in response.css("article").extract_first():
    print(i.css("div.texts h2 a::attr(href)").extract_first())
# print(response.xpath("/html/body/div[5]/section/div/div[3]/div[3]/div").extract_first())

# response.xpath("//item/title").extract_first()

TypeError: 'NoneType' object is not iterable

In [45]:
from scrapy.crawler import CrawlerProcess
from scrapy.utils.project import get_project_settings

process = CrawlerProcess(get_project_settings())

process.crawl('your-spider')
process.start() # the script will block here until the crawling is finished

2020-09-13 12:23:53 [scrapy.utils.log] INFO: Scrapy 2.3.0 started (bot: scrapybot)
2020-09-13 12:23:53 [scrapy.utils.log] INFO: Versions: lxml 4.5.2.0, libxml2 2.9.10, cssselect 1.1.0, parsel 1.6.0, w3lib 1.22.0, Twisted 20.3.0, Python 3.7.7 (default, May  7 2020, 21:25:33) - [GCC 7.3.0], pyOpenSSL 19.1.0 (OpenSSL 1.1.1g  21 Apr 2020), cryptography 3.1, Platform Linux-5.4.0-42-generic-x86_64-with-debian-buster-sid
2020-09-13 12:23:53 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.epollreactor.EPollReactor


KeyError: 'Spider not found: your-spider'

In [46]:
import scrapy
from scrapy.crawler import CrawlerProcess

class MySpider(scrapy.Spider):
    # Your spider definition
    ...

process = CrawlerProcess(settings={
    "FEEDS": {
        "items.json": {"format": "json"},
    },
})

process.crawl(MySpider)
process.start() # the script will block here until the crawling is finished

2020-09-13 12:28:48 [scrapy.utils.log] INFO: Scrapy 2.3.0 started (bot: scrapybot)
2020-09-13 12:28:48 [scrapy.utils.log] INFO: Versions: lxml 4.5.2.0, libxml2 2.9.10, cssselect 1.1.0, parsel 1.6.0, w3lib 1.22.0, Twisted 20.3.0, Python 3.7.7 (default, May  7 2020, 21:25:33) - [GCC 7.3.0], pyOpenSSL 19.1.0 (OpenSSL 1.1.1g  21 Apr 2020), cryptography 3.1, Platform Linux-5.4.0-42-generic-x86_64-with-debian-buster-sid
2020-09-13 12:28:48 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.epollreactor.EPollReactor
2020-09-13 12:28:48 [scrapy.crawler] INFO: Overridden settings:
{}
2020-09-13 12:28:48 [scrapy.extensions.telnet] INFO: Telnet Password: 4ee000aa933ad962
2020-09-13 12:28:48 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.memusage.MemoryUsage',
 'scrapy.extensions.feedexport.FeedExporter',
 'scrapy.extensions.logstats.LogStats']


In [47]:
import scrapy

class BlogSpider(scrapy.Spider):
    name = 'blogspider'
    start_urls = ['https://blog.scrapinghub.com']

    def parse(self, response):
        for title in response.css('.post-header>h2'):
            yield {'title': title.css('a ::text').get()}

        for next_page in response.css('a.next-posts-link'):
            yield response.follow(next_page, self.parse)